In [4]:
import sqlite3
import pandas as pd

connection = sqlite3.connect('../datasets/checking-logs.sqlite')
str10 =pd.io.sql.read_sql('SELECT * FROM test LIMIT 10;',connection)

# print(str10)

query_test = """SELECT time, AVG(diff) as avg_diff
                FROM (SELECT uid,labname, AVG((strftime('%s', first_commit_ts) - deadlines)/ 3600) as diff,
                CASE 
                    WHEN (strftime('%s', first_commit_ts) -strftime('%s', first_view_ts))/3600 < 0
                        THEN 'before'
                    ELSE 'after'
                END AS time
                FROM test
                LEFT JOIN deadlines
                ON test.labname = deadlines.labs
                WHERE labname != 'project1'
                GROUP BY uid,time)            
                WHERE uid IN 
                (
                SELECT uid FROM 
                (SELECT uid,
                CASE 
                    WHEN (strftime('%s', first_commit_ts) -strftime('%s', first_view_ts))/3600 < 0
                        THEN 'before'
                    ELSE 'after'
                END AS time
                FROM test
                LEFT JOIN deadlines
                ON test.labname = deadlines.labs
                WHERE labname != 'project1'
                GROUP BY uid,time)
                GROUP BY uid
                HAVING COUNT (uid)==2)
                GROUP BY time"""

query_control = """SELECT time, AVG(diff) as avg_diff
            FROM (SELECT uid,labname, AVG((strftime('%s', first_commit_ts) - deadlines)/ 3600) as diff,
            CASE 
                WHEN (strftime('%s', first_commit_ts) -strftime('%s', first_view_ts))/3600 < 0
                    THEN 'before'
                ELSE 'after'
            END AS time
            FROM control
            LEFT JOIN deadlines
            ON control.labname = deadlines.labs
            WHERE labname != 'project1'
            GROUP BY uid,time)            
            WHERE uid IN 
            (
            SELECT uid FROM 
            (SELECT uid,
            CASE 
                WHEN (strftime('%s', first_commit_ts) -strftime('%s', first_view_ts))/3600 < 0
                    THEN 'before'
                ELSE 'after'
            END AS time
            FROM control
            LEFT JOIN deadlines
            ON control.labname = deadlines.labs
            WHERE labname != 'project1'
            GROUP BY uid,time)
            GROUP BY uid
            HAVING COUNT (uid)==2)
            GROUP BY time"""

test_results= pd.io.sql.read_sql(query_test,connection)

control_results= pd.io.sql.read_sql(query_control,connection)


print(test_results)
print(control_results)

connection.close()


     time   avg_diff
0   after -99.523810
1  before -66.047619
     time   avg_diff
0   after -99.322222
1  before -98.033333
